# [KTO]_DrCard_analysis_v1_2210

*<strong>분석 개요</strong>*

    A. 지점별 * 일별 디지털 주민증 신청 현황 

In [1]:
# basic
import gc
import os
import sys
import warnings
warnings.filterwarnings(action='ignore') 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#--------------------#
# handling
#--------------------#
import math
import time
import random
import importlib
import xlsxwriter
import numpy as np
import pandas as pd
from collections import Counter
from datetime import datetime, timedelta

#--------------------#
# Vis
#--------------------#
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
## Vis for jupyter theme
from IPython.display import Image
from jupyterthemes import jtplot
jtplot.style(theme= 'grade3', context='notebook', ticks=True, grid=False) ## dark backgroud jupyter notebook theme
# plt.style.use(['seaborn-white']) ## dark background style ## white style : 'seaborn-white'
plt.style.use(['dark_background']) ## dark background style ## white style : 'seaborn-white'
## Vis for korean 
import matplotlib.font_manager as fm
font_location = '/usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf'
fprop = fm.FontProperties(fname=font_location)
font_name = fprop.get_name()
matplotlib.rc('font', family=font_name)

In [2]:
#--------------------#
# sphere package
#--------------------#
sys.path.append("/home/das_share/sphere_class/")
import SpherePackage
from SpherePackage import *
for pkg in [SpherePackage] :
    _ = importlib.reload(pkg)

#--------------------#
# kto package
#--------------------#
sys.path.append("../src/")
import kto_config
import kto_util
import kto_prep
from kto_config import *
from kto_prep import *
from kto_util import *

for pkg in [kto_config, kto_prep, kto_util] :
    _ = importlib.reload(pkg)

# 0. config

# 1.Data import

In [3]:
#------------------------------------------------#
# 0. config
#------------------------------------------------#
## 1) 데이터 기간 세팅
yesterday = str(datetime.today().date() - timedelta(days = 1))  # auto setting yesrterday using0 today
s_date = '20220926'
e_date = yesterday.split('-')[0] + yesterday.split('-')[1] + yesterday.split('-')[2]
today = datetime(int(e_date[:4]),int(e_date[4:6]),int(e_date[6:8])) # today = datetime(2021,5,6)
dates = (datetime.strptime(e_date, '%Y%m%d') - datetime.strptime(s_date, '%Y%m%d')).days + 1 

## 2) parameter
store_file_nm = '앱(App) 유입.20221001_20221004-혜택명칭 체크_수정.xlsx'
store_file_sheet_nm = '배너 INDEX'

EVENT_DEEPLINK = 'deepLInk'
PARAM_QR_STORE_CODE = 'params_qrStoreCode'

ABSOLUTE_OUTPUT_PATH = '/home/th/analysis/analysis_report/00_custom_analysis/kto_custom/data/out/DrCard_output/'
DRCARD_OUTPUT_FILE_NM = 'DrCard_daily_regit_by_store_{}.xlsx'.format(str(today.date()).replace('-',''))


In [4]:
#------------------------------------------------#
# 1. Data import
#------------------------------------------------#
## 1) log data 
df_app_log = DataImport.read_files(
    input_path = INPUT_PATH + DATA_TYPE + APP_KEY, 
    today = today + timedelta(days = 1), 
    dates = dates,
    platform_total = True, reduce_memory = True, verbose = False)
df_app_log.date.min()
df_app_log.date.max()

## 1) store file data
df_store_file = pd.read_excel(f'../data/in/{store_file_nm}', sheet_name=store_file_sheet_nm)
df_store_file = df_store_file.rename(columns = {'QR 순번' : PARAM_QR_STORE_CODE})
df_store_file

'2022-09-26 00:00:01'

'2022-11-09 23:59:58'

,지역,params_qrStoreCode,장소명
0,평창,pc01,스페이스 창공
1,평창,pc02,평창군(관광두레 전반)
2,평창,pc03,평창캠프닉
3,평창,pc04,오대산 힐링빌리지
4,평창,pc05,어름치마을
5,평창,pc06,방림별곡카페
6,평창,pc07,옐로우트리카페
7,평창,pc08,평창사랑
8,평창,pc09,너나드리
9,평창,pc10,대관령마켓


# 2.Preprocess

In [5]:
_, df_param_contents = DataImport.json_to_dataframe_nodeN(
    df_app_log, key_id=[KEY_ID_DEVICE, KEY_ID_USER],
     type = 'manual', lst_manual_events= EVENT_CONTENT_DETAIL, 
    )

In [6]:
#------------------------------------------------#
# 1. prep
#------------------------------------------------#
## 1) log
df_app_log = SpherePrep.BasicPrep.basic_prep(df_app_log)

## 2) param
_, df_param_all = DataImport.json_to_dataframe_nodeN(
    df_app_log, key_id=[KEY_ID_DEVICE, KEY_ID_USER],
     type = 'manual', lst_manual_events= EVENT_DEEPLINK, 
    )
    
#------------------------------------------------#
# 2. param detail data
#------------------------------------------------#
df_param_deeplink = DataImport.extract_event_param(df_param_all, EVENT_DEEPLINK).drop_duplicates()
df_param_deeplink[PARAM_QR_STORE_CODE] =\
    df_param_deeplink[PARAM_QR_STORE_CODE].apply(lambda x : x.lower() if isinstance(x, str) else x)

# 3.Analysis

In [7]:
#------------------------------------------------#
# 3. analysis
#------------------------------------------------#
## 1) store daily
df_store_daily = df_param_deeplink.groupby([PARAM_QR_STORE_CODE, 'day']).size().to_frame().reset_index()
df_store_daily_wide = df_store_daily.pivot(index='day', columns = PARAM_QR_STORE_CODE,  values=0).fillna(0)
df_store_daily_wide = df_store_daily_wide[sorted([x for x in df_store_daily_wide.columns if x in df_store_file[PARAM_QR_STORE_CODE].unique()])]
## 2) sum total
df_store_daily_wide_sum_total = df_store_daily_wide.sum().reset_index().\
    pivot_table(columns = PARAM_QR_STORE_CODE, values = 0).rename(index = {0 : 'total'})
df_store_daily_wide = pd.concat(
    [df_store_daily_wide,df_store_daily_wide_sum_total])
df_store_daily_wide

params_qrStoreCode,oc01,oc02,oc03,oc04,pc01,pc02,pc03,pc04,pc05,pc06,pc07,pc08,pc09,pc10,pc11
2022-09-29,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-09-30,3.0,15.0,0.0,1.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-01,0.0,3.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,20.0,22.0,67.0,0.0
2022-10-02,0.0,2.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,30.0,28.0,46.0,0.0
2022-10-03,1.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,51.0,27.0,14.0,0.0
2022-10-04,3.0,13.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,7.0,3.0,0.0
2022-10-05,62.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,4.0,4.0,0.0
2022-10-06,79.0,5.0,0.0,0.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,3.0,2.0,1.0,0.0
2022-10-07,19.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,5.0,0.0,0.0
2022-10-08,7.0,55.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,4.0,5.0,7.0,14.0,0.0


In [8]:
#------------------------------------------------#
# 5. multi columns
#------------------------------------------------#
lst_store_code = list(df_store_daily_wide.columns)
df_store_file_target = df_store_file[df_store_file[PARAM_QR_STORE_CODE].isin(lst_store_code)]
lst_store_loc = list(df_store_file_target['지역'].values)
lst_store_nm = list(df_store_file_target['장소명'].values)
df_store_file
lst_store_code
lst_store_loc
lst_store_nm

,지역,params_qrStoreCode,장소명
0,평창,pc01,스페이스 창공
1,평창,pc02,평창군(관광두레 전반)
2,평창,pc03,평창캠프닉
3,평창,pc04,오대산 힐링빌리지
4,평창,pc05,어름치마을
5,평창,pc06,방림별곡카페
6,평창,pc07,옐로우트리카페
7,평창,pc08,평창사랑
8,평창,pc09,너나드리
9,평창,pc10,대관령마켓


['oc01',
 'oc02',
 'oc03',
 'oc04',
 'pc01',
 'pc02',
 'pc03',
 'pc04',
 'pc05',
 'pc06',
 'pc07',
 'pc08',
 'pc09',
 'pc10',
 'pc11']

['평창',
 '평창',
 '평창',
 '평창',
 '평창',
 '평창',
 '평창',
 '평창',
 '평창',
 '평창',
 '평창',
 '옥천',
 '옥천',
 '옥천',
 '옥천']

['스페이스 창공',
 '평창군(관광두레 전반)',
 '평창캠프닉',
 '오대산 힐링빌리지',
 '어름치마을',
 '방림별곡카페',
 '옐로우트리카페',
 '평창사랑',
 '너나드리',
 '대관령마켓',
 '산너미목장',
 '수생식물학습원',
 '옥천전통문화체험관',
 '장령산자연휴양림',
 '달빛마을']

In [9]:
#------------------------------------------------#
# 3. analysis
#------------------------------------------------#
## 1) store daily
df_store_daily = df_param_deeplink.groupby([PARAM_QR_STORE_CODE, 'day']).size().to_frame().reset_index()
df_store_daily_wide = df_store_daily.pivot(index='day', columns = PARAM_QR_STORE_CODE,  values=0).fillna(0)
df_store_daily_wide = df_store_daily_wide[[x for x in df_store_file[PARAM_QR_STORE_CODE].unique() if x in df_store_daily_wide.columns]]
## 2) sum total
df_store_daily_wide_sum_total = df_store_daily_wide.sum().reset_index().\
    pivot_table(columns = PARAM_QR_STORE_CODE, values = 0).rename(index = {0 : 'total'})
df_store_daily_wide = pd.concat(
    [df_store_daily_wide,df_store_daily_wide_sum_total])

#------------------------------------------------#
# 5. multi columns
#------------------------------------------------#
lst_store_code = list(df_store_daily_wide.columns)
df_store_file_target = df_store_file[df_store_file[PARAM_QR_STORE_CODE].isin(lst_store_code)]
lst_store_loc = list(df_store_file_target['지역'].values)
lst_store_nm = list(df_store_file_target['장소명'].values)

df_store_daily_wide.columns = [
    lst_store_loc,
    lst_store_nm,
    lst_store_code
    ]
df_store_daily_wide
#------------------------------------------------#
# 5. save output
#------------------------------------------------#
df_store_daily_wide.to_excel(ABSOLUTE_OUTPUT_PATH + DRCARD_OUTPUT_FILE_NM)

평창                                                           \
           스페이스 창공 평창군(관광두레 전반) 평창캠프닉 오대산 힐링빌리지 어름치마을 방림별곡카페 옐로우트리카페   평창사랑   
              pc01         pc02  pc03      pc04  pc05   pc06    pc07   pc08   
2022-09-29    19.0          0.0   0.0       0.0   0.0    0.0     0.0    0.0   
2022-09-30    30.0          0.0   0.0       0.0   0.0    0.0     0.0    0.0   
2022-10-01     0.0          0.0   6.0       0.0   0.0    0.0     0.0   20.0   
2022-10-02     0.0          0.0   6.0       0.0   0.0    0.0     0.0   30.0   
2022-10-03     3.0          0.0   1.0       0.0   0.0    0.0     0.0   51.0   
2022-10-04    10.0          0.0   0.0       0.0   0.0    0.0     0.0    4.0   
2022-10-05     0.0          0.0   1.0       0.0   0.0    0.0     1.0    1.0   
2022-10-06     4.0          1.0   0.0       0.0   2.0    0.0     0.0    3.0   
2022-10-07     0.0          0.0   0.0       1.0   0.0    2.0     0.0    0.0   
2022-10-08     0.0          0.0   1.0       0.0   0.0    1.0     4.0    5.0   
2022-10-09     8.0          0.0   4.0       1.0   0.0    0.0     6.0    3.0   
2022-10-10     1.0          0.0   4.0       0.0   0.0    0.0     1.0    0.0   
2022-10-11     1.0          0.0   0.0       0.0   0.0    0.0     3.0    2.0   
2022-10-12     1.0          0.0   0.0       0.0   0.0    0.0     2.0    1.0   
2022-10-13    16.0          0.0   0.0       0.0   0.0    0.0     0.0    1.0   
2022-10-14    27.0          0.0   0.0       0.0   0.0    0.0     1.0    0.0   
2022-10-15     3.0          0.0   0.0       0.0   4.0    0.0     0.0    0.0   
2022-10-16     0.0          0.0   0.0       0.0   0.0    0.0     0.0    0.0   
2022-10-17    21.0          0.0   0.0       0.0   0.0    0.0     2.0    0.0   
2022-10-18     5.0          0.0   0.0       0.0   0.0    0.0     0.0    0.0   
2022-10-19     1.0          0.0   0.0       0.0   0.0    4.0     0.0    0.0   
2022-10-20     0.0          0.0   0.0       0.0  30.0    0.0     0.0    0.0   
2022-10-21     0.0          0.0   0.0       0.0   0.0    0.0     1.0    1.0   
2022-10-22     0.0          0.0   1.0       0.0   0.0    0.0     1.0   76.0   
2022-10-23     2.0          0.0   0.0       0.0   0.0    0.0     0.0    0.0   
2022-10-24     4.0          0.0   0.0       0.0   0.0    0.0     2.0    0.0   
2022-10-25     3.0          0.0   0.0       0.0   0.0    0.0     0.0    0.0   
2022-10-26     0.0          0.0   0.0       0.0   0.0    0.0     2.0    0.0   
2022-10-27     2.0          0.0   0.0       0.0   0.0    1.0     2.0    0.0   
2022-10-28    21.0          0.0   0.0       0.0   0.0    7.0     4.0    0.0   
2022-10-29     7.0          0.0   0.0       0.0   0.0    0.0     2.0    0.0   
2022-10-30     8.0          0.0   0.0       0.0   0.0    0.0     1.0    0.0   
2022-10-31    24.0          0.0   0.0       0.0   0.0    0.0     0.0    0.0   
2022-11-01    19.0          0.0   0.0       0.0   0.0    1.0     0.0    0.0   
2022-11-02    21.0          0.0   0.0       0.0   0.0    0.0     0.0    0.0   
2022-11-03   117.0          0.0   0.0       0.0   0.0    2.0     0.0    0.0   
2022-11-04    18.0          0.0   0.0       0.0   0.0    0.0     2.0    7.0   
2022-11-05     0.0          0.0   2.0       0.0   0.0    0.0     7.0    9.0   
2022-11-06     0.0          0.0  16.0       1.0   0.0    0.0     2.0    0.0   
2022-11-07    39.0          0.0   4.0       0.0   0.0    0.0     0.0    0.0   
2022-11-08     0.0          0.0   0.0       0.0  14.0    0.0     0.0    0.0   
2022-11-09     4.0          0.0   9.0       0.0   0.0    0.0     1.0    0.0   
total        439.0          1.0  55.0       3.0  50.0   18.0    47.0  214.0   

                                     옥천                           
             너나드리  대관령마켓  산너미목장 수생식물학습원 옥천전통문화체험관 장령산자연휴양림  달빛마을  
             pc09   pc10   pc11    oc01      oc02     oc03  oc04  
2022-09-29    0.0    0.0    0.0     0.0       0.0      0.0   0.0  
2022-09-30    0.0    0.0    0.0     3.0      15.0      0.0   1.0  
2022-10-01   22.0   67.0    0.0     0.0       

In [35]:
def DrCardMain() :
    #------------------------------------------------#
    # 0. config
    #------------------------------------------------#
    ## 1) 데이터 기간 세팅
    yesterday = str(datetime.today().date() - timedelta(days = 1))  # auto setting yesrterday using0 today
    s_date = '20220926'
    e_date = yesterday.split('-')[0] + yesterday.split('-')[1] + yesterday.split('-')[2]
    today = datetime(int(e_date[:4]),int(e_date[4:6]),int(e_date[6:8])) # today = datetime(2021,5,6)
    dates = (datetime.strptime(e_date, '%Y%m%d') - datetime.strptime(s_date, '%Y%m%d')).days + 1 

    ## 2) parameter
    store_file_nm = '앱(App) 유입.20221001_20221004-혜택명칭 체크_수정.xlsx'
    store_file_sheet_nm = '배너 INDEX'

    EVENT_DEEPLINK = 'deepLInk'
    PARAM_QR_STORE_CODE = 'params_qrStoreCode'

    ABSOLUTE_INPUT_PATH = '/home/th/analysis/analysis_report/00_custom_analysis/kto_custom/data/in/'
    ABSOLUTE_OUTPUT_PATH = '/home/th/analysis/analysis_report/00_custom_analysis/kto_custom/data/out/DrCard_output/'
    DRCARD_OUTPUT_FILE_NM = 'DrCard_daily_regit_by_store_{}.xlsx'.format(str(today.date()).replace('-',''))


    #------------------------------------------------#
    # 1. log Data import
    #------------------------------------------------#
    ## 1) log file data
    df_app_log = DataImport.read_files(
        input_path = INPUT_PATH + DATA_TYPE + APP_KEY, 
        today = today + timedelta(days = 1), 
        dates = dates,
        platform_total = True, reduce_memory = True, verbose = False)
    ## 2) store file data
    df_store_file = pd.read_excel(f'{ABSOLUTE_INPUT_PATH}{store_file_nm}', sheet_name=store_file_sheet_nm)
    df_store_file = df_store_file.rename(columns = {'QR 순번' : PARAM_QR_STORE_CODE})

    #------------------------------------------------#
    # 2. prep
    #------------------------------------------------#
    ## 1) log
    df_app_log = SpherePrep.BasicPrep.basic_prep(df_app_log)

    ## 2) param
    _, df_param_all = DataImport.json_to_dataframe_nodeN(
        df_app_log, key_id=[KEY_ID_DEVICE, KEY_ID_USER],
        type = 'manual', lst_manual_events= EVENT_DEEPLINK, 
        )
        
    ## 3) param detail data
    df_param_deeplink = DataImport.extract_event_param(df_param_all, EVENT_DEEPLINK).drop_duplicates()
    df_param_deeplink[PARAM_QR_STORE_CODE] =\
        df_param_deeplink[PARAM_QR_STORE_CODE].apply(lambda x : x.lower() if isinstance(x, str) else x)


    #------------------------------------------------#
    # 3. analysis
    #------------------------------------------------#
    ## 1) store daily
    df_store_daily = df_param_deeplink.groupby([PARAM_QR_STORE_CODE, 'day']).size().to_frame().reset_index()
    df_store_daily_wide = df_store_daily.pivot(index='day', columns = PARAM_QR_STORE_CODE,  values=0).fillna(0)
    df_store_daily_wide = df_store_daily_wide[[x for x in df_store_file[PARAM_QR_STORE_CODE].unique() if x in df_store_daily_wide.columns]]
    ## 2) sum total
    df_store_daily_wide_sum_total = df_store_daily_wide.sum().reset_index().\
        pivot_table(columns = PARAM_QR_STORE_CODE, values = 0).rename(index = {0 : 'total'})
    df_store_daily_wide = pd.concat(
        [df_store_daily_wide,df_store_daily_wide_sum_total])

    #------------------------------------------------#
    # 4. multi columns
    #------------------------------------------------#
    lst_store_code = list(df_store_daily_wide.columns)
    df_store_file_target = df_store_file[df_store_file[PARAM_QR_STORE_CODE].isin(lst_store_code)]
    lst_store_loc = list(df_store_file_target['지역'].values)
    lst_store_nm = list(df_store_file_target['장소명'].values)

    df_store_daily_wide.columns = [
        lst_store_loc,
        lst_store_nm,
        lst_store_code
        ]
    df_store_daily_wide
    #------------------------------------------------#
    # 5. save output
    #------------------------------------------------#
    df_store_daily_wide.to_excel(ABSOLUTE_OUTPUT_PATH + DRCARD_OUTPUT_FILE_NM)

    return df_store_daily_wide